In [5]:
"""
1호선 열차운행시각표 Plotly 인터랙티브 시각화 (Jupyter Notebook용)
※ 주피터 노트북에서 셀 단위로 실행하세요
"""

# =============================================================================
# 📦 라이브러리 임포트 및 데이터 로드
# =============================================================================
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 데이터 로드 (파일 경로는 환경에 맞게 수정)
df = pd.read_csv(
    r'data\(1호선)서울교통공사_서울 도시철도 열차운행시각표_20250704.csv',
    encoding='euc-kr'
)

# 시간 데이터 변환
df['열차도착시간'] = pd.to_datetime(df['열차도착시간'], format='%H:%M:%S', errors='coerce')
df['열차출발시간'] = pd.to_datetime(df['열차출발시간'], format='%H:%M:%S', errors='coerce')
df['시간대'] = df['열차출발시간'].dt.hour
df['열차구분'] = df['급행여부'].map({0: '일반', 1: '급행'})

print(f"✅ 데이터 로드 완료: {df.shape}")
print(f"📊 총 {df['역사명'].nunique()}개 역, {df['열차코드'].nunique()}개 열차")

✅ 데이터 로드 완료: (88431, 14)
📊 총 102개 역, 1019개 열차


In [6]:
# =============================================================================
# 📊 1. 시간대별 운행 빈도 (막대 그래프 + 출퇴근 시간 강조)
# =============================================================================
time_freq = df['시간대'].value_counts().sort_index().reset_index()
time_freq.columns = ['시간대', '운행횟수']

# 출퇴근 시간대 구분
time_freq['구분'] = time_freq['시간대'].apply(
    lambda x: '출근시간(7-9시)' if 7 <= x <= 9 
    else '퇴근시간(18-20시)' if 18 <= x <= 20 
    else '일반시간'
)

fig1 = px.bar(
    time_freq,
    x='시간대',
    y='운행횟수',
    color='구분',
    title='🚇 1호선 시간대별 열차 운행 빈도',
    labels={'시간대': '시간대 (시)', '운행횟수': '운행 횟수 (회)'},
    color_discrete_map={
        '일반시간': '#3498db',
        '출근시간(7-9시)': '#f39c12',
        '퇴근시간(18-20시)': '#e74c3c'
    },
    text='운행횟수'
)

fig1.update_traces(texttemplate='%{text}', textposition='outside')
fig1.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),
    height=500,
    hovermode='x unified'
)

fig1.show()

In [7]:
# =============================================================================
# 📈 2. 주중/주말/공휴일 시간대별 비교 (라인 차트)
# =============================================================================
period_data = df.groupby(['주중주말', '시간대']).size().reset_index(name='운행횟수')

# 주중주말 이름 매핑
period_map = {'DAY': '주중', 'SAT': '토요일', 'END': '공휴일'}
period_data['구분'] = period_data['주중주말'].map(period_map)

fig2 = px.line(
    period_data,
    x='시간대',
    y='운행횟수',
    color='구분',
    title='📅 주중/주말/공휴일 시간대별 운행 패턴 비교',
    labels={'시간대': '시간대 (시)', '운행횟수': '운행 횟수 (회)'},
    markers=True,
    line_shape='spline'
)

fig2.update_layout(
    xaxis=dict(tickmode='linear', tick0=0, dtick=2),
    height=500,
    hovermode='x unified'
)

fig2.show()

In [8]:
# =============================================================================
# 🚄 3. 시간대별 급행/일반 열차 분포 (스택 영역 차트)
# =============================================================================
hourly_type = df.groupby(['시간대', '열차구분']).size().unstack(fill_value=0)

fig3 = go.Figure()

# 일반 열차
fig3.add_trace(go.Scatter(
    x=hourly_type.index,
    y=hourly_type['일반'],
    mode='lines',
    name='일반 열차',
    line=dict(width=0.5, color='rgb(111, 184, 224)'),
    stackgroup='one',
    fillcolor='rgba(111, 184, 224, 0.7)',
    hovertemplate='시간대: %{x}시<br>일반: %{y}회<extra></extra>'
))

# 급행 열차
fig3.add_trace(go.Scatter(
    x=hourly_type.index,
    y=hourly_type['급행'],
    mode='lines',
    name='급행 열차',
    line=dict(width=0.5, color='rgb(234, 99, 140)'),
    stackgroup='one',
    fillcolor='rgba(234, 99, 140, 0.7)',
    hovertemplate='시간대: %{x}시<br>급행: %{y}회<extra></extra>'
))

fig3.update_layout(
    title='🚄 시간대별 급행/일반 열차 운행 분포',
    xaxis_title='시간대 (시)',
    yaxis_title='운행 횟수 (회)',
    hovermode='x unified',
    height=500
)

fig3.show()

In [9]:
# =============================================================================
# 🏆 4. 운행 횟수 TOP 15 역 (수평 막대 그래프)
# =============================================================================
top_stations = df['역사명'].value_counts().head(15).reset_index()
top_stations.columns = ['역사명', '운행횟수']
top_stations = top_stations.sort_values('운행횟수', ascending=True)

fig4 = px.bar(
    top_stations,
    y='역사명',
    x='운행횟수',
    orientation='h',
    title='🏆 1호선 운행 횟수 TOP 15 역',
    labels={'역사명': '역명', '운행횟수': '운행 횟수 (회)'},
    text='운행횟수',
    color='운행횟수',
    color_continuous_scale='Blues'
)

fig4.update_traces(texttemplate='%{text:,}회', textposition='outside')
fig4.update_layout(height=600, showlegend=False)

fig4.show()

In [10]:
# =============================================================================
# 🗺️ 5. 출발역-도착역 노선 히트맵
# =============================================================================
# 상위 10개 출발역과 도착역
top_starts = df['출발역'].value_counts().head(10).index.tolist()
top_ends = df['도착역'].value_counts().head(10).index.tolist()

# 노선별 운행 횟수 집계
route_matrix = []
for start in top_starts:
    row = []
    for end in top_ends:
        count = len(df[(df['출발역'] == start) & (df['도착역'] == end)])
        row.append(count)
    route_matrix.append(row)

fig5 = go.Figure(data=go.Heatmap(
    z=route_matrix,
    x=top_ends,
    y=top_starts,
    colorscale='YlOrRd',
    text=route_matrix,
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="운행 횟수"),
    hovertemplate='출발: %{y}<br>도착: %{x}<br>운행: %{z}회<extra></extra>'
))

fig5.update_layout(
    title='🗺️ 주요 출발역-도착역 노선 운행 현황',
    xaxis_title='도착역',
    yaxis_title='출발역',
    height=600
)

fig5.show()

In [11]:
# =============================================================================
# 📈 6. 급행 비율 시간대별 변화 (라인 + 영역 차트)
# =============================================================================
express_ratios = []
hours = []

for hour in range(5, 24):
    hour_df = df[df['시간대'] == hour]
    if len(hour_df) > 0:
        express_count = len(hour_df[hour_df['급행여부'] == 1])
        total_count = len(hour_df)
        express_ratio = express_count / total_count * 100
        
        hours.append(hour)
        express_ratios.append(express_ratio)

fig6 = go.Figure()

# 라인 + 마커 + 영역
fig6.add_trace(go.Scatter(
    x=hours,
    y=express_ratios,
    mode='lines+markers',
    name='급행 비율',
    line=dict(color='crimson', width=3),
    marker=dict(size=10, symbol='circle', color='crimson'),
    fill='tozeroy',
    fillcolor='rgba(220, 20, 60, 0.2)',
    hovertemplate='%{x}시: %{y:.1f}%<extra></extra>'
))

# 20% 기준선
fig6.add_hline(
    y=20, 
    line_dash="dash", 
    line_color="gray",
    annotation_text="20% 기준선",
    annotation_position="right"
)

fig6.update_layout(
    title='📈 시간대별 급행 열차 운행 비율 변화',
    xaxis_title='시간대 (시)',
    yaxis_title='급행 비율 (%)',
    hovermode='x unified',
    height=500
)

fig6.show()

In [12]:
# =============================================================================
# 📊 7. 배차간격 분포 (박스플롯)
# =============================================================================
major_stations = ['서울역', '용산', '영등포', '구로', '신도림', '노량진']
interval_data = []

for station in major_stations:
    station_df = df[
        (df['역사명'] == station) & 
        (df['주중주말'] == 'DAY') &
        (df['시간대'] >= 7) & 
        (df['시간대'] < 9) &
        (df['열차출발시간'].notna())
    ].copy()
    
    station_df = station_df.sort_values('열차출발시간')
    station_df['배차간격_분'] = station_df['열차출발시간'].diff().dt.total_seconds() / 60
    
    for _, row in station_df.dropna(subset=['배차간격_분']).iterrows():
        interval_data.append({
            '역명': station,
            '배차간격(분)': row['배차간격_분']
        })

interval_df = pd.DataFrame(interval_data)

fig7 = px.box(
    interval_df,
    x='역명',
    y='배차간격(분)',
    title='📊 주요 역 배차간격 분포 (주중 출근시간 7-9시)',
    labels={'역명': '역명', '배차간격(분)': '배차 간격 (분)'},
    color='역명',
    points='all'
)

fig7.update_layout(height=500, showlegend=False)

fig7.show()

In [13]:
# =============================================================================
# 🎯 8. 종합 대시보드 (4개 차트를 하나로)
# =============================================================================
fig8 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '시간대별 운행 빈도',
        '주중/주말 비교',
        '급행/일반 비율',
        '방향별 운행'
    ),
    specs=[
        [{"type": "bar"}, {"type": "scatter"}],
        [{"type": "pie"}, {"type": "bar"}]
    ]
)

# 1. 시간대별 운행 빈도
time_freq_simple = df['시간대'].value_counts().sort_index()
fig8.add_trace(
    go.Bar(x=time_freq_simple.index, y=time_freq_simple.values, 
           name='운행빈도', marker_color='steelblue'),
    row=1, col=1
)

# 2. 주중/주말 비교
for period, color, name in [('DAY', 'blue', '주중'), ('SAT', 'orange', '토요일'), ('END', 'red', '공휴일')]:
    period_df = df[df['주중주말'] == period]
    hourly = period_df['시간대'].value_counts().sort_index()
    fig8.add_trace(
        go.Scatter(x=hourly.index, y=hourly.values, name=name,
                  line=dict(color=color), mode='lines+markers'),
        row=1, col=2
    )

# 3. 급행/일반 비율 (파이 차트)
train_types = df['열차구분'].value_counts()
fig8.add_trace(
    go.Pie(labels=train_types.index, values=train_types.values,
           marker=dict(colors=['lightblue', 'coral'])),
    row=2, col=1
)

# 4. 방향별 운행
directions = df['방향'].value_counts()
fig8.add_trace(
    go.Bar(x=directions.index, y=directions.values, name='방향별',
           marker=dict(color=['green', 'purple'])),
    row=2, col=2
)

fig8.update_layout(
    title_text='🎯 1호선 열차운행 종합 대시보드',
    showlegend=True,
    height=800
)

fig8.show()

In [14]:
# =============================================================================
# ⏰ 9. 시간대별 애니메이션 차트
# =============================================================================
major_stations_anim = ['서울역', '용산', '영등포', '구로', '신도림', '노량진', '종각', '동대문']

animation_data = []
for hour in range(5, 24):
    for station in major_stations_anim:
        count = len(df[(df['역사명'] == station) & (df['시간대'] == hour)])
        animation_data.append({
            '시간대': hour,
            '역명': station,
            '운행횟수': count
        })

animation_df = pd.DataFrame(animation_data)

fig9 = px.bar(
    animation_df,
    x='역명',
    y='운행횟수',
    animation_frame='시간대',
    title='⏰ 시간대별 주요 역 운행 변화 (▶ 재생 버튼 클릭)',
    labels={'역명': '역명', '운행횟수': '운행 횟수 (회)', '시간대': '시간대'},
    color='운행횟수',
    color_continuous_scale='Viridis',
    range_y=[0, animation_df['운행횟수'].max() + 10]
)

fig9.update_layout(height=600)

fig9.show()


# =============================================================================
# ✅ 완료 메시지
# =============================================================================
print("\n" + "="*70)
print("✅ 모든 Plotly 인터랙티브 시각화 완료!")
print("="*70)
print("\n💡 각 차트는 인터랙티브합니다:")
print("  - 마우스 호버: 상세 정보 표시")
print("  - 확대/축소: 드래그 또는 줌 버튼 사용")
print("  - 범례 클릭: 데이터 계열 표시/숨김")
print("  - 애니메이션: 재생 버튼으로 시간대별 변화 확인")


✅ 모든 Plotly 인터랙티브 시각화 완료!

💡 각 차트는 인터랙티브합니다:
  - 마우스 호버: 상세 정보 표시
  - 확대/축소: 드래그 또는 줌 버튼 사용
  - 범례 클릭: 데이터 계열 표시/숨김
  - 애니메이션: 재생 버튼으로 시간대별 변화 확인
